In [1]:
import requests

In [2]:
url = 'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios'

headers = {'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'}
c = requests.get(url, headers = headers).content

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(c)

## Criando uma lista com os preços dos carros

In [13]:
lista_precos_tags = soup.find_all('p', {'class':"eoKYee"})

- For loop para obter a lista de preços

In [16]:
# lista_precos = []
# for tag in lista_precos_tags:
#     lista_precos.append(tag.text)
# lista_precos

 - List comprehension

In [18]:
# for loop utilizando list compreehension
lista_precos  = [tag.text for tag in lista_precos_tags]
lista_precos[0:10]

['R$ 108.000',
 'R$ 33.987',
 'R$ 15.900',
 'R$ 84.290',
 'R$ 85.000',
 'R$ 119.900',
 'R$ 83.990',
 'R$ 33.900',
 'R$ 45.900',
 'R$ 47.890']

## Criando uma lista com algumas caraterísticas do modelo.

In [22]:
lista_caracteristicas_tags = soup.find_all('div', {'class':'sc-bwzfXH'})

In [26]:
lista_caracteristicas = [info.text for info in lista_caracteristicas_tags ]
lista_caracteristicas[0:5]

['',
 'Procurar pelo título do anúncioBusca por categoriasTodas as categoriasAutos e peçasCarros, vans e utilitáriosMarcaCITROENFIATFORDGM - CHEVROLETHONDAHYUNDAIJACKIA MOTORSNISSANPEUGEOTRENAULTTOYOTAVW - VOLKSWAGENACURAAGRALEALFA ROMEOAM GENAMERICARASIA MOTORSASTON MARTINAUDIBABYBMWBRMBUGGYBUGRECADILLACCBT JIPECHANACHANGANCHERYCHRYSLERCROSS LANDERDAEWOODAIHATSUDKW VEMAGDODGEEFFAENGESAENVEMOFERRARIFIBRAVANFOTONFYBERGEELYGREAT WALLGURGELHAFEIHITECH ELETRICINFINITIISUZUIVECOJAGUARJEEPJINBEIJPXLADALAMBORGHINILAND ROVERLANDWINDLEXUSLIFANLOBINILOTUSMAHINDRAMASERATIMATRAMAZDAMCLARENMERCEDES-BENZMERCURYMGMINIMITSUBISHIMIURAPLYMOUTHPONTIACPORSCHEPUMARAMRELYROLLS-ROYCEROVERSAABSATURNSEATSHINERAYSMARTSSANGYONGSUBARUSUZUKITACTESLATROLLERVOLVOWAKEWALKModeloQuilometragemKm de05.00010.00020.00030.00040.00060.00080.000100.000120.000140.000160.000180.000200.000250.000300.000400.000500.000Km até05.00010.00020.00030.00040.00060.00080.000100.000120.000140.000160.000180.000200.000250.000300.000400.000500

## Criando uma lista com os endereços

In [ ]:
lista_enderecos_tags = soup.find_all('span' , {'class':'ui-search-item__group__element'})

In [ ]:
lista_enderecos = [end.text for end in lista_enderecos_tags]
lista_enderecos = lista_enderecos[1::2]
lista_enderecos[:5]

## Criando  função que retorna um DataFrame com as informaçoes obtidas anteriormente

In [ ]:
zonas = ['norte', 'sul', 'leste', 'oeste']
URL='https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', 
           '_Desde_241', '_Desde_289']

In [ ]:
def scrap(url):
    c = requests.get(url).content
    soup = BeautifulSoup(c)
    lista_precos_tags = soup.find_all('span', {'class':'price-tag-fraction'})
    lista_precos  = [tag.text for tag in lista_precos_tags]
    lista_area_quartos_tags = soup.find_all('ul', {'class':'ui-search-card-attributes'})
    lista_area_quartos = [info.text for info in lista_area_quartos_tags ]
    lista_enderecos_tags = soup.find_all('span' , {'class':'ui-search-item__group__element'})
    lista_enderecos = [end.text for end in lista_enderecos_tags]
    lista_enderecos = lista_enderecos[1::2]
    df=convert_df(zona, lista_area_quartos, lista_enderecos, lista_precos)
    return df


def convert_df(zona, lista_area_quartos, lista_enderecos, lista_precos):
    zonas_lista = [zona]*len(lista_precos)
    data = {'zona': zonas_lista,
            'area_quartos': lista_area_quartos,
            'enderecos_lista':lista_enderecos,
            'aluguel': lista_precos}
    df = pd.DataFrame(data)
    return df

In [ ]:
lista_dfs = []
for zona in zonas:
    for pag in paginas:
        url='https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-{}/{}'.format(zona,pag)
        print(url)
        df=scrap(url)
        lista_dfs.append(df)
        sleep(1)

In [ ]:
df = pd.concat(lista_dfs)

In [ ]:
df.head()

In [ ]:
df.to_csv('dados_aluguels_imoveis_ml_sao_paulo.csv', index =False , encoding = 'latin-1')